In [1]:
import pandas as pd
from sympy import true

def correct_timestamp(ts):
    if ts[11:13] == "24":
        ts = ts[:11] + "00" + ts[13:]  # Replace '24' with '00'
        corrected_datetime = pd.to_datetime(ts, format="%Y/%m/%d/%H/%M/%S")
    else:
        corrected_datetime = pd.to_datetime(ts, format="%Y/%m/%d/%H/%M/%S")
    return corrected_datetime

def get_event_tables(search_result, site_search, after_search_product_click, product_view):
    
    site_search["event_timestamp"] = site_search["event_timestamp"].astype(str)
    site_search["event_timestamp"] = site_search["event_timestamp"].fillna("2000/01/01/01/01/01")
    site_search["event_timestamp"] = site_search["event_timestamp"].apply(correct_timestamp)
    
    search_result["event_timestamp"] = search_result["event_timestamp"].astype(str)
    search_result["event_timestamp"] = search_result["event_timestamp"].fillna("2000/01/01/01/01/01")
    search_result["event_timestamp"] = search_result["event_timestamp"].apply(correct_timestamp)


    after_search_product_click["event_timestamp"] = after_search_product_click["event_timestamp"].astype(str)
    after_search_product_click["event_timestamp"] = after_search_product_click["event_timestamp"].fillna("2000/01/01/01/01/01")
    after_search_product_click["event_timestamp"] = after_search_product_click["event_timestamp"].apply(correct_timestamp)
    
    # Merge site_search and search_result
    site_search_result_df = pd.merge(
        site_search,
        search_result,
        on=["customer_id", "ga_session_id", "event_timestamp"],
        how="left"
    )
    # Initialize lists for matched and unmatched
    matched_rows = []
    # Improve matching logic
    for after_row in after_search_product_click.itertuples(index=False):
        condition = (
            (site_search_result_df["customer_id"] == after_row.customer_id)
            & (site_search_result_df["ga_session_id"] == after_row.ga_session_id)
            & (site_search_result_df["event_timestamp"] < after_row.event_timestamp)
        )
        filtered_site_search = site_search_result_df[condition]
        if not filtered_site_search.empty:
            closest_row = filtered_site_search.iloc[
                -1
            ].copy()  
            for col in after_search_product_click.columns:
                closest_row[f"after_{col}"] = getattr(after_row, col)
            matched_rows.append(closest_row)
    matched_df = pd.DataFrame(matched_rows)
    
    # Mark matched rows so that unmatched rows can be easily filtered
    matched_indices = matched_df.index
    unmatched_df = site_search_result_df.loc[
        ~site_search_result_df.index.isin(matched_indices)
    ].copy()
    for col in after_search_product_click.columns:
        unmatched_df[f"after_{col}"] = None
        
    # Suppress FutureWarning
    matched_df.dropna(axis=1, how='all', inplace=True)
    unmatched_df.dropna(axis=1, how='all', inplace=True)
        
    # Merge matched and unmatched DataFrames
    site_search_result_after = pd.concat(
        [matched_df, unmatched_df], ignore_index=True
    )
    # Split product_view's event_value
    product_view[["product_name", "product_sku"]] = (
        product_view["event_value"]
        .str.split("\|\|\|", expand=True)
        .apply(lambda x: x.str.strip())
    )
    # Perform final merge and rename
    site_search_result_after_view = pd.merge(
        site_search_result_after,
        product_view[["product_name", "product_sku"]].drop_duplicates(),
        how="left",
        left_on="after_event_value",
        right_on="product_name",
    )
    reformat_site_search_result_after_view = site_search_result_after_view.rename(
        columns={
            "event_value_x": "search_term",
            "event_value_y": "search_result",
            "report_date_x": "report_date",
            "cookie_id_x": "cookie_id",
            "after_event_value": "product_name",
            "after_event_timestamp": "after_search_click_timestamp",
        }
    )
    columns_to_drop = [
        "event_name_x",
        "report_date_y",
        "cookie_id_y",
        "event_name_y",
        "after_report_date",
        "after_cookie_id",
        "after_customer_id",
        "after_ga_session_id",
        "after_event_name",
    ]
    reformat_site_search_result_after_view = (
        reformat_site_search_result_after_view.drop(columns=columns_to_drop)
    )
    reformat_site_search_result_after_view = (
        reformat_site_search_result_after_view.sort_values(
            by="event_timestamp", ascending=True
        ).reset_index(drop=True)
    )
    
    return reformat_site_search_result_after_view

In [2]:
search_result = pd.read_csv("Event - Search Result.csv")
site_search = pd.read_csv("Event - Site Search.csv")
after_search_product_click = pd.read_csv("Event - After Search Product Click.csv")
product_view = pd.read_csv("Event - Product View.csv")
A = get_event_tables(search_result, site_search, after_search_product_click, product_view)

In [3]:
A.to_csv("linked.csv")